In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glucocorticoid_Sensitivity"
cohort = "GSE32962"

# Input paths
in_trait_dir = "../../input/GEO/Glucocorticoid_Sensitivity"
in_cohort_dir = "../../input/GEO/Glucocorticoid_Sensitivity/GSE32962"

# Output paths
out_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/GSE32962.csv"
out_gene_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE32962.csv"
out_clinical_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/clinical_data/GSE32962.csv"
json_path = "../../output/preprocess/Glucocorticoid_Sensitivity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"In vitro prednisolone resistance signature in MLL-rearranged infant ALL"
!Series_summary	"Acute Lymphoblastic Leukemia (ALL) in infants (<1 year of age) is characterized by a high incidence of MLL translocations which is associated with a poor prognosis. Contributing to this poor prognosis is cellular drug resistance, especially to glucocorticoids like prednisolone. Although in vitro prednisolone resistance mechanisms have been proposed in pediatric ALL, it has never been studied in MLL-rearranged infant ALL, which are highly resistant to glucocorticoids in vitro and in vivo."
!Series_overall_design	"We analyzed primary MLL-rearranged infant ALL expression profiles, which were either in vitro prednisolone-resistant or prednisolone-sensitive, in order to study in vitro prednisolone resistance."
Sample Characteristics Dictionary:
{0: ['disease state: Precursor B-cell acute lymphoblastic leukemia (ALL)'], 1: ['cell type: Primary infant ALL cells at d

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on background information mentioning ALL expression profiles
# This indicates gene expression data should be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From sample characteristics, identifying keys for trait, age, and gender
trait_row = 4  # prednisolone sensitivity
age_row = 3    # age
gender_row = None  # gender information is not available

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert prednisolone sensitivity to binary value."""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'resistant' in value:
        return 1  # resistant
    elif 'sensitive' in value:
        return 0  # sensitive
    else:
        return None

def convert_age(value):
    """Convert age information to a standardized format."""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # All patients are < 1 year of age, so we can't get specific ages
    # Will return constant value since all are infants
    if '< 1 year' in value.lower():
        return 0.5  # Approximate midpoint for infants under 1 year
    else:
        return None

# 3. Save Metadata - Initial Filtering
# Check if trait information is available (based on trait_row not being None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Assuming clinical_data is available as a variable from a previous step
    # Let's try to access it directly first, rather than loading from a file
    try:
        # Try to access clinical_data from the environment
        if 'clinical_data' in globals():
            clinical_df = clinical_data
        else:
            # If not available directly, try loading from predefined paths
            try:
                clinical_df = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.txt"), sep='\t')
            except FileNotFoundError:
                # Look for other potential files that might contain the clinical data
                potential_files = [f for f in os.listdir(in_cohort_dir) if os.path.isfile(os.path.join(in_cohort_dir, f))]
                
                if 'clinical.csv' in potential_files:
                    clinical_df = pd.read_csv(os.path.join(in_cohort_dir, 'clinical.csv'))
                elif 'characteristics.csv' in potential_files:
                    clinical_df = pd.read_csv(os.path.join(in_cohort_dir, 'characteristics.csv'))
                else:
                    # If we can't find the clinical data, we may need to skip this step
                    print(f"Could not find clinical data files in {in_cohort_dir}")
                    print(f"Available files: {potential_files}")
                    raise FileNotFoundError("Clinical data not found. Cannot proceed with clinical feature extraction.")
        
        # Extract clinical features using the library function
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_df,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=None
        )
        
        # Preview the resulting dataframe
        preview = preview_df(selected_clinical_df)
        print("Clinical Data Preview:")
        print(preview)
        
        # Save the clinical data
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error during clinical data processing: {str(e)}")
        print("Skipping clinical feature extraction step.")


Clinical Data Preview:
{'GSM816393': [1.0, 0.5], 'GSM816394': [1.0, 0.5], 'GSM816395': [1.0, 0.5], 'GSM816396': [1.0, 0.5], 'GSM816397': [1.0, 0.5], 'GSM816398': [1.0, 0.5], 'GSM816399': [1.0, 0.5], 'GSM816400': [1.0, 0.5], 'GSM816401': [1.0, 0.5], 'GSM816402': [1.0, 0.5], 'GSM816403': [1.0, 0.5], 'GSM816404': [1.0, 0.5], 'GSM816405': [1.0, 0.5], 'GSM816406': [1.0, 0.5], 'GSM816407': [1.0, 0.5], 'GSM816408': [1.0, 0.5], 'GSM816409': [1.0, 0.5], 'GSM816410': [1.0, 0.5], 'GSM816411': [1.0, 0.5], 'GSM816412': [1.0, 0.5], 'GSM816413': [1.0, 0.5], 'GSM816414': [1.0, 0.5], 'GSM816415': [1.0, 0.5], 'GSM816416': [1.0, 0.5], 'GSM816417': [0.0, 0.5], 'GSM816418': [0.0, 0.5], 'GSM816419': [0.0, 0.5], 'GSM816420': [0.0, 0.5], 'GSM816421': [0.0, 0.5], 'GSM816422': [0.0, 0.5], 'GSM816423': [0.0, 0.5], 'GSM816424': [0.0, 0.5], 'GSM816425': [0.0, 0.5], 'GSM816426': [0.0, 0.5], 'GSM816427': [0.0, 0.5], 'GSM816428': [0.0, 0.5], 'GSM816429': [0.0, 0.5], 'GSM816430': [0.0, 0.5], 'GSM816431': [0.0, 0.5], '

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 64
Header line: "ID_REF"	"GSM816393"	"GSM816394"	"GSM816395"	"GSM816396"	"GSM816397"	"GSM816398"	"GSM816399"	"GSM816400"	"GSM816401"	"GSM816402"	"GSM816403"	"GSM816404"	"GSM816405"	"GSM816406"	"GSM816407"	"GSM816408"	"GSM816409"	"GSM816410"	"GSM816411"	"GSM816412"	"GSM816413"	"GSM816414"	"GSM816415"	"GSM816416"	"GSM816417"	"GSM816418"	"GSM816419"	"GSM816420"	"GSM816421"	"GSM816422"	"GSM816423"	"GSM816424"	"GSM816425"	"GSM816426"	"GSM816427"	"GSM816428"	"GSM816429"	"GSM816430"	"GSM816431"	"GSM816432"	"GSM816433"	"GSM816434"	"GSM816435"
First data line: "1007_s_at"	3.907563631	3.318962591	3.98536714	4.271082363	2.662877522	4.922713597	5.212086782	3.542450029	5.171343838	4.097394296	4.474937028	4.427429853	4.155954785	3.839779638	4.479720696	3.371124677	4.038207496	4.219455083	3.892608822	4.45604985	4.402967007	3.704188659	3.89026041	4.076764483	4.167769501	3.908383635	4.230898849	4.00237022	4.325269426	3.644105352	4.879649423	5.40470506	4.495447925	4.594496346	5

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers like "1007_s_at", "1053_at", "117_at", etc.
# These appear to be Affymetrix probe IDs from a microarray platform,
# not standard human gene symbols like BRCA1, TP53, etc.
# Therefore, these identifiers need to be mapped to standard gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE32962
Line 6: !Series_title = In vitro prednisolone resistance signature in MLL-rearranged infant ALL
Line 7: !Series_geo_accession = GSE32962
Line 8: !Series_status = Public on Oct 13 2011
Line 9: !Series_submission_date = Oct 13 2011
Line 10: !Series_last_update_date = Mar 25 2019
Line 11: !Series_pubmed_id = 22282267
Line 12: !Series_summary = Acute Lymphoblastic Leukemia (ALL) in infants (<1 year of age) is characterized by a high incidence of MLL translocations which is associated with a poor prognosis. Contributing to this poor prognosis is cellular drug resistance, especially to glucocorticoids like prednisolone. Although in vitro prednisolone resistance mechanisms have been proposed in pediatric 


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in gene annotation that match the gene IDs and gene symbols
probe_id_column = 'ID'  # Observed from gene annotation data - matches the index of gene_data
gene_symbol_column = 'Gene Symbol'  # Column containing gene symbols

# 2. Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, probe_id_column, gene_symbol_column)

# Preview the mapping to ensure it's correct
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply the mapping to convert probe-level expression to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Check the first few rows and shape of the resulting gene expression data
print("\nGene data after mapping:")
print(f"Shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the processed gene data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A

Gene data after mapping:
Shape: (21253, 43)
First few gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE32962.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_data = pd.read_csv(out_clinical_data_file)
print(f"Clinical data shape: {clinical_data.shape}")
print("Clinical data preview:")
print(clinical_data.head())

# 3. Link clinical and genetic data
# First convert clinical data to proper format with trait as column name
clinical_df_t = clinical_data.T.reset_index()
clinical_df_t.columns = ['Sample_ID', trait, 'Age']  # Set proper column names

# Convert gene expression data to have samples as rows
gene_data_t = normalized_gene_data.T.reset_index()
gene_data_t = gene_data_t.rename(columns={'index': 'Sample_ID'})

# Merge the two dataframes on Sample_ID
linked_data = pd.merge(clinical_df_t, gene_data_t, on='Sample_ID')
linked_data = linked_data.set_index('Sample_ID')

print(f"\nLinked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 10 columns):")
if linked_data.shape[1] >= 10:
    print(linked_data.iloc[:5, :10])
else:
    print(linked_data.head())

# 4. Handle missing values
if linked_data.shape[0] > 0:
    print("\nMissing values before handling:")
    print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
    
    gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
    if gene_cols:
        missing_genes_pct = linked_data[gene_cols].isna().mean()
        genes_with_high_missing = sum(missing_genes_pct > 0.2)
        print(f"  Genes with >20% missing: {genes_with_high_missing}")
        
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

    # Handle missing values
    cleaned_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {cleaned_data.shape}")

    # 5. Evaluate bias in trait and demographic features
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
else:
    print("No samples in linked data, skipping missing value handling and bias evaluation")
    cleaned_data = linked_data
    trait_biased = True  # Mark as biased if we have no data

# 6. Final validation and save
note = "Dataset contains gene expression data from glucocorticoid sensitivity studies. "
if linked_data.shape[0] > 0:
    if 'Age' in cleaned_data.columns:
        note += "Age data is available but constant across all samples (all patients are infants under 1 year). "
else:
    note += "ERROR: No samples remained after linking clinical and gene data."

is_gene_available = len(normalized_gene_data) > 0
is_trait_available = True

# Check if the cleaned data is empty
if cleaned_data.shape[0] == 0:
    note += " WARNING: Dataset has 0 samples after preprocessing."
    
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (19823, 43)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Gene data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE32962.csv
Clinical data shape: (2, 43)
Clinical data preview:
   GSM816393  GSM816394  GSM816395  GSM816396  GSM816397  GSM816398  \
0        1.0        1.0        1.0        1.0        1.0        1.0   
1        0.5        0.5        0.5        0.5        0.5        0.5   

   GSM816399  GSM816400  GSM816401  GSM816402  ...  GSM816426  GSM816427  \
0        1.0        1.0        1.0        1.0  ...        0.0        0.0   
1        0.5        0.5        0.5        0.5  ...        0.5        0.5   

   GSM816428  GSM816429  GSM816430  GSM816431  GSM816432  GSM816433  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.5        0.5        0.5        0.5        0.5        0.5   

   GSM816434  GSM816435  
0        0.0        0.0  
1        0.5        0.5  

[2 rows x 43 columns]

Linked data shape: (43, 19825)
Linked data preview (first 5 rows, first 10 columns):
           

Data shape after handling missing values: (43, 19825)
For the feature 'Glucocorticoid_Sensitivity', the least common label is '0.0' with 19 occurrences. This represents 44.19% of the dataset.
The distribution of the feature 'Glucocorticoid_Sensitivity' in this dataset is fine.

Quartiles for 'Age':
  25%: 0.5
  50% (Median): 0.5
  75%: 0.5
Min: 0.5
Max: 0.5
The distribution of the feature 'Age' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/GSE32962.csv
